In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, ElasticNet

In [23]:
df = pd.read_csv("/content/drive/MyDrive/BRAIN_CANCER (1).csv")
df.head()
df.info()
df.isna().sum()
categorical_columns = df.select_dtypes(include=['object'])
print(categorical_columns)
df=df.drop(['ID','KPS','PsP_TP_score'],axis=1)
print(df['Survival_from_surgery_days'].unique())
df = df[df['Survival_from_surgery_days'] != 'Not Available']
df = df.drop(df.columns[[0,1]], axis=1)
df = pd.get_dummies(df, columns=['Gender', 'IDH1', 'MGMT', 'GTR_over90percent'])
df['Survival_from_surgery_days'] = pd.to_numeric(df['Survival_from_surgery_days'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Columns: 1740 entries, Unnamed: 0.1 to PsP_TP_score
dtypes: float64(1167), int64(566), object(7)
memory usage: 8.0+ MB
                     ID Gender Survival_from_surgery_days      IDH1  \
0    UPENN-GBM-00002_11      F                        291  Wildtype   
1    UPENN-GBM-00003_11      M              Not Available  Wildtype   
2    UPENN-GBM-00005_11      M              Not Available  Wildtype   
3    UPENN-GBM-00006_11      M                        626  Wildtype   
4    UPENN-GBM-00007_11      M              Not Available  Wildtype   
..                  ...    ...                        ...       ...   
595  UPENN-GBM-00607_11      F              Not Available  Wildtype   
596  UPENN-GBM-00608_11      F              Not Available  Wildtype   
597  UPENN-GBM-00609_11      M                        138  Wildtype   
598  UPENN-GBM-00610_11      M              Not Available   Mutated   
599  UPENN-GBM-00611_11     

In [29]:
df = df.fillna(df.mean())


In [32]:
from scipy.stats import zscore

# Compute the z-scores of numeric features
z_scores = zscore(df)

# Identify outliers based on a threshold
threshold = 2
outliers = np.abs(z_scores) > threshold

# Remove outliers by filtering the DataFrame
df = df[~outliers]

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

# Fit the imputer on the data
imputer.fit(df)

# Impute missing values
df = pd.DataFrame(imputer.transform(df), columns=df.columns)


In [33]:
df.head()

,T1_ED_Intensity_CoefficientOfVariation,T1_ED_Intensity_Energy,T1_ED_Intensity_InterQuartileRange,T1_ED_Intensity_Kurtosis,T1_ED_Intensity_Maximum,T1_ED_Intensity_Mean,T1_ED_Intensity_MeanAbsoluteDeviation,T1_ED_Intensity_Median,T1_ED_Intensity_MedianAbsoluteDeviation,T1_ED_Intensity_Minimum,...,IDH1_Mutated,IDH1_NOS/NEC,IDH1_Wildtype,MGMT_Indeterminate,MGMT_Methylated,MGMT_Not Available,MGMT_Unmethylated,GTR_over90percent_N,GTR_over90percent_Not Available,GTR_over90percent_Y
0,0.166209,1.063257e+09,34.0,3.214771,235.0,147.193603,-3.150000e-12,145.0,2.193603,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.150632,7.654425e+08,24.0,3.344714,197.0,120.907615,-4.050000e-12,118.0,2.907615,50.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.083001,7.632825e+08,18.0,3.922620,214.0,170.916185,1.560000e-12,171.0,-0.083815,84.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.153633,1.834986e+09,25.0,3.438159,213.0,125.606402,1.221882e-13,123.0,2.606402,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.136548,1.475727e+09,25.0,3.447739,217.0,137.263974,-1.020000e-13,133.0,0.655683,53.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [34]:
from sklearn.model_selection import train_test_split

# Separate the target variable and the features
y = df['Survival_from_surgery_days']
X = df.drop(columns=['Survival_from_surgery_days'])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
from sklearn.feature_selection import SelectKBest, f_regression

# Select the top k features based on F-test
k = 12
selector = SelectKBest(score_func=f_regression, k=k)
x_train_selected = selector.fit_transform(X_train, y_train)
x_test_selected = selector.transform(X_test)

selected_indices = selector.get_support(indices=True)
selected_columns = X_train.columns[selected_indices]
print(selected_columns)

Index(['T1_ET_Morphologic_EllipseDiameter_Axis-2',
       'T2_ED_Histogram_Bins-16_Bins-16_Median',
       'T2_ED_Histogram_Bins-16_Bins-16_SeventyFifthPercentile',
       'T2_ET_Morphologic_EllipseDiameter_Axis-2',
       'FLAIR_ET_Histogram_Bins-16_Bins-16_Bin-15_Probability',
       'FLAIR_ET_Morphologic_EllipseDiameter_Axis-2',
       'FLAIR_NC_GLRLM_Bins-16_Radius-1_ShortRunEmphasis',
       'T1GD_ET_Morphologic_EllipseDiameter_Axis-2',
       'T1GD_ET_GLCM_Bins-16_Radius-1_ClusterProminence', 'Age_at_scan_years',
       'GTR_over90percent_N', 'GTR_over90percent_Y'],
      dtype='object')


In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)

# Fit the GridSearchCV object to find the best hyperparameters
grid_search.fit(x_train_selected, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_


from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100)
rf.fit(x_train_selected, y_train)

# Gradient Boosting Regressor
gb = GradientBoostingRegressor(n_estimators=100)
gb.fit(x_train_selected, y_train)


GradientBoostingRegressor()

In [43]:
from sklearn.preprocessing import RobustScaler

# Create a scaler object
scaler = RobustScaler()

# Fit the scaler on the training data
scaler.fit(x_train_selected)

# Scale the features
X_train_scaled = scaler.transform(x_train_selected)
X_test_scaled = scaler.transform(x_test_selected)


In [45]:
X_train_scaled.shape

(356, 12)

In [44]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Create an SVR model
model = SVR()

# Train the model on the scaled training data
model.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test_scaled)




Mean Squared Error: 33613.9370237869
R-squared Score: 0.032214727709378055


In [ ]:


max_value = df['T1_ET_Morphologic_EllipseDiameter_Axis-2'].max()
min_value = df['T1_ET_Morphologic_EllipseDiameter_Axis-2'].min()

print("Maximum value of 'T1_ET_Morphologic_EllipseDiameter_Axis-2':", max_value)
print("Minimum value of 'T1_ET_Morphologic_EllipseDiameter_Axis-2':", min_value)

max_value = df['T2_ED_Histogram_Bins-16_Bins-16_Median'].max()
min_value = df['T2_ED_Histogram_Bins-16_Bins-16_Median'].min()

print("Maximum value of 'T2_ED_Histogram_Bins-16_Bins-16_Median':", max_value)
print("Minimum value of 'T2_ED_Histogram_Bins-16_Bins-16_Median':", min_value)

max_value = df['T2_ET_Histogram_Bins-16_Bins-16_Bin-14_Probability'].max()
min_value = df['T2_ET_Histogram_Bins-16_Bins-16_Bin-14_Probability'].min()

print("Maximum value of 'T2_ET_Histogram_Bins-16_Bins-16_Bin-14_Probability':", max_value)
print("Minimum value of 'T2_ET_Histogram_Bins-16_Bins-16_Bin-14_Probability':", min_value)

max_value = df['T2_ET_Histogram_Bins-16_Bins-16_Bin-15_Probability'].max()
min_value = df['T2_ET_Histogram_Bins-16_Bins-16_Bin-15_Probability'].min()

print("Maximum value of 'T2_ET_Histogram_Bins-16_Bins-16_Bin-15_Probability':", max_value)
print("Minimum value of 'T2_ET_Histogram_Bins-16_Bins-16_Bin-15_Probability':", min_value)

max_value = df['T2_ET_Morphologic_EllipseDiameter_Axis-2'].max()
min_value = df['T2_ET_Morphologic_EllipseDiameter_Axis-2'].min()

print("Maximum value of 'T2_ET_Morphologic_EllipseDiameter_Axis-2':", max_value)
print("Minimum value of 'T2_ET_Morphologic_EllipseDiameter_Axis-2':", min_value)

max_value = df['FLAIR_ET_Morphologic_EllipseDiameter_Axis-2'].max()
min_value = df['FLAIR_ET_Morphologic_EllipseDiameter_Axis-2'].min()

print("Maximum value of 'FLAIR_ET_Morphologic_EllipseDiameter_Axis-2':", max_value)
print("Minimum value of 'FLAIR_ET_Morphologic_EllipseDiameter_Axis-2':", min_value)

max_value = df['FLAIR_NC_GLRLM_Bins-16_Radius-1_LongRunEmphasis'].max()
min_value = df['FLAIR_NC_GLRLM_Bins-16_Radius-1_LongRunEmphasis'].min()

print("Maximum value of 'FLAIR_NC_GLRLM_Bins-16_Radius-1_LongRunEmphasis':", max_value)
print("Minimum value of 'FLAIR_NC_GLRLM_Bins-16_Radius-1_LongRunEmphasis':", min_value)

max_value = df['FLAIR_NC_GLRLM_Bins-16_Radius-1_ShortRunEmphasis'].max()
min_value = df['FLAIR_NC_GLRLM_Bins-16_Radius-1_ShortRunEmphasis'].min()

print("Maximum value of 'FLAIR_NC_GLRLM_Bins-16_Radius-1_ShortRunEmphasis':", max_value)
print("Minimum value of 'FLAIR_NC_GLRLM_Bins-16_Radius-1_ShortRunEmphasis':", min_value)

max_value = df['T1GD_ET_Morphologic_EllipseDiameter_Axis-2'].max()
min_value = df['T1GD_ET_Morphologic_EllipseDiameter_Axis-2'].min()

print("Maximum value of 'T1GD_ET_Morphologic_EllipseDiameter_Axis-2':", max_value)
print("Minimum value of 'T1GD_ET_Morphologic_EllipseDiameter_Axis-2':", min_value)

max_value = df['GTR_over90percent_N'].max()
min_value = df['GTR_over90percent_N'].min()

print("Maximum value of 'GTR_over90percent_N':", max_value)
print("Minimum value of 'GTR_over90percent_N':", min_value)

max_value = df['GTR_over90percent_Y'].max()
min_value = df['GTR_over90percent_Y'].min()

print("Maximum value of: 'GTR_over90percent_Y':", max_value)
print("Maximum value of: 'GTR_over90percent_Y':", min_value)


In [48]:
new_data = RobustScaler.transform([[50, 2, 0.005, 0.009799118, 60, 58, 1.2, 0.95, 57, 80, 54, 47]])
predicted_values = model.predict(new_data)
print(predicted_values)


TypeError: ignored